Notebook made for completing our assignment


# Project Proposal

### Introduction

Diabetes is a metabolic disease disallowing the use or creation (depending on the type of diabetes) of insulin. This disease impedes various metabolic functions and can result in fatal consequences if left untreated. It is therefore imperative for strong predictive measures to be implemented to ensure early identification of the disease. For this reason, our project hopes to answer the predictive question of “how do variables such as plasma glucose concentration, blood pressure, and BMI predict whether an individual has Diabetes or not?” 


The dataset we will be using to answer this question is the Pima Indians Diabetes Dataset that was created through data collected by the National Institute of Diabetes and Digestive and Kidney Diseases. This dataset has been constrained to only women of at least 21 years of age and of Pima Indian heritage with the goal of isolating the dataset from as many confounding variables as possible. It contains various medical predictors of diabetes (including skin thickness, glucose concentration, bmi, number of pregnancies, blood pressure, insulin levels, and the diabetes pedigree function) and one boolean outcome variable.


### Preliminary exploratory data analysis

In [ ]:
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier

In [2]:
diabetes=pd.read_csv('data/diabetes.csv')

In [3]:
diabetes

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


### Methods

In our data analysis, we have chosen to use **scikit-learn's k-nearest neighbours (KNN) classification** algorithm as a pivotal component of our predictive modelling strategy. Our initial steps involve loading and exploring the Pima Indians Diabetes Dataset using pandas, where we will address any **missing values** or data cleaning requirements using a **preprocessor**. 


To guide our **variable selection**, we will employ a preliminary analysis, utilizing data **visualizations** and **correlation matrices**. These visualizations will assist in identifying patterns, relationships, and potential predictors that may significantly contribute to diabetes prediction. For example, to compare **blood pressure** between individuals with and without diabetes, we will use **boxplots. These visualizations will display the distribution of blood pressure values for each group, allowing for a quick comparison of central tendency and spread. The boxplots will help identify potential differences in blood pressure that may be indicative of its relevance as a predictive variable. 


Subsequently, we will split the dataset into training and testing sets and train the KNN classifier using the training data. Model evaluation will be conducted using metrics such as accuracy score on the testing set. By adopting this approach, we aim to build a robust and effective model for early identification of diabetes in Pima Indian women, employing the strengths of the KNN algorithm in the classification task.
